# GoogleGenerativeAIEmbeddings

This will help you get started with Google Generative AI [embedding models](/docs/concepts/embedding_models) using LangChain. For detailed documentation on `GoogleGenerativeAIEmbeddings` features and configuration options, please refer to the [API reference](https://api.js.langchain.com/classes/langchain_google_genai.GoogleGenerativeAIEmbeddings.html).

## Overview
### Integration details

| Class | Package | Local | [Py support](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [`GoogleGenerativeAIEmbeddings`](https://api.js.langchain.com/classes/langchain_google_genai.GoogleGenerativeAIEmbeddings.html) | [`@langchain/google-genai`](https://npmjs.com/@langchain/google-genai) | ❌ | ✅ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/google-genai?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/google-genai?style=flat-square&label=%20&) |

## Setup

To access Google Generative AI embedding models you'll need to sign up for a Google AI account, get an API key, and install the `@langchain/google-genai` integration package.

### Credentials

Get an API key here: https://ai.google.dev/tutorials/setup.

Next, set your key as an environment variable named `GOOGLE_API_KEY`:

```bash
export GOOGLE_API_KEY="your-api-key"
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```bash
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="your-api-key"
```

### Installation

The LangChain `GoogleGenerativeAIEmbeddings` integration lives in the `@langchain/google-genai` package. You may also wish to install the official SDK:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/google-genai @langchain/core @google/generative-ai
</Npm2Yarn>
```

## Instantiation

Now we can instantiate our model object and embed text:

In [1]:
import { GoogleGenerativeAIEmbeddings } from "@langchain/google-genai";
import { TaskType } from "@google/generative-ai";

const embeddings = new GoogleGenerativeAIEmbeddings({
  model: "text-embedding-004", // 768 dimensions
  taskType: TaskType.RETRIEVAL_DOCUMENT,
  title: "Document title",
});

## Indexing and Retrieval

Embedding models are often used in retrieval-augmented generation (RAG) flows, both as part of indexing data as well as later retrieving it. For more detailed instructions, please see our RAG tutorials under the [working with external knowledge tutorials](/docs/tutorials/#working-with-external-knowledge).

Below, see how to index and retrieve data using the `embeddings` object we initialized above. In this example, we will index and retrieve a sample document using the demo [`MemoryVectorStore`](/docs/integrations/vectorstores/memory).

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## Direct Usage

Under the hood, the vectorstore and retriever implementations are calling `embeddings.embedDocument(...)` and `embeddings.embedQuery(...)` to create embeddings for the text(s) used in `fromDocuments` and the retriever's `invoke` operations, respectively.

You can directly call these methods to get embeddings for your own use cases.

### Embed single texts

You can embed queries for search with `embedQuery`. This generates a vector representation specific to the query:

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
  -0.018286658,   0.020051053,  -0.057487167,   0.0059406986, -0.0036901247,
  -0.010400916,    0.03396853,  -0.010867519,    0.015650319,   0.026443942,
   0.012251757,   -0.01581729,    0.02031182, -0.00062176475,  0.0065521155,
   -0.07107355,   0.033614952,    0.07109807,   -0.021078493,   0.048039366,
   0.022973344,    -0.0361746,   -0.04550704,   -0.048807852,    0.03414146,
   0.042450827,    0.02930612,   0.027274853,   -0.027707053,   -0.04167595,
    0.01708843,   0.028532283, -0.0018593844,      -0.096786,  -0.034648854,
  0.0013152987,   0.024425535,    0.04937838,    0.036890924,  -0.074619934,
  -0.028723065,   0.029158255,  -0.023993572,     0.03163398,   -0.02036324,
   -0.02333609,  -0.017407075, -0.0059643993,    -0.05564625,   0.051022638,
    0.03264913,  -0.008254581,  -0.030552095,    0.072952054,   -0.05448913,
   0.012030814,   -0.07978849,  -0.030417662,   0.0038343794,    0.03237516,
  -0.054259773,    -0.0524064,   -0.02145499,    0.006439614,    0.0498894

### Embed multiple texts

You can embed multiple texts for indexing with `embedDocuments`. The internals used for this method may (but do not have to) differ from embedding queries:

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
  -0.018286658,   0.020051053,  -0.057487167,   0.0059406986, -0.0036901247,
  -0.010400916,    0.03396853,  -0.010867519,    0.015650319,   0.026443942,
   0.012251757,   -0.01581729,    0.02031182, -0.00062176475,  0.0065521155,
   -0.07107355,   0.033614952,    0.07109807,   -0.021078493,   0.048039366,
   0.022973344,    -0.0361746,   -0.04550704,   -0.048807852,    0.03414146,
   0.042450827,    0.02930612,   0.027274853,   -0.027707053,   -0.04167595,
    0.01708843,   0.028532283, -0.0018593844,      -0.096786,  -0.034648854,
  0.0013152987,   0.024425535,    0.04937838,    0.036890924,  -0.074619934,
  -0.028723065,   0.029158255,  -0.023993572,     0.03163398,   -0.02036324,
   -0.02333609,  -0.017407075, -0.0059643993,    -0.05564625,   0.051022638,
    0.03264913,  -0.008254581,  -0.030552095,    0.072952054,   -0.05448913,
   0.012030814,   -0.07978849,  -0.030417662,   0.0038343794,    0.03237516,
  -0.054259773,    -0.0524064,   -0.02145499,    0.006439614,    0.0498894

## API reference

For detailed documentation of all `GoogleGenerativeAIEmbeddings` features and configurations head to the API reference: https://api.js.langchain.com/classes/langchain_google_genai.GoogleGenerativeAIEmbeddings.html